<a href="https://colab.research.google.com/github/romerofernandes/Car_consumption/blob/main/Car_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importações

In [111]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt

In [112]:
pd.set_option('display.max_rows', None)

In [113]:
all_csv_files = glob.glob("/content/*.csv")
all_csv_files

['/content/skoda.csv',
 '/content/hyundi.csv',
 '/content/unclean focus.csv',
 '/content/vauxhall.csv',
 '/content/merc.csv',
 '/content/audi.csv',
 '/content/vw.csv',
 '/content/bmw.csv',
 '/content/toyota.csv',
 '/content/focus.csv',
 '/content/ford.csv',
 '/content/unclean cclass.csv',
 '/content/cclass.csv']

In [114]:
df_car = pd.concat(map(pd.read_csv, all_csv_files), axis=0, ignore_index=True)

In [115]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   model         117995 non-null  object 
 1   year          117903 non-null  float64
 2   price         117995 non-null  object 
 3   transmission  117995 non-null  object 
 4   mileage       117077 non-null  object 
 5   fuelType      108540 non-null  object 
 6   tax           94327 non-null   float64
 7   mpg           99187 non-null   float64
 8   engineSize    108540 non-null  float64
 9   tax(£)        4860 non-null    float64
 10  fuel type     3517 non-null    object 
 11  engine size   9345 non-null    object 
 12  mileage2      9399 non-null    object 
 13  fuel type2    8537 non-null    object 
 14  engine size2  8537 non-null    object 
 15  reference     9455 non-null    object 
dtypes: float64(5), object(11)
memory usage: 14.4+ MB


#Transformações

In [116]:
df_car.dropna(subset=['mpg'], how='any', inplace=True)

In [117]:
df_car.dropna(axis='columns', inplace=True)

In [118]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 0 to 110244
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99187 non-null  object 
 1   year          99187 non-null  float64
 2   price         99187 non-null  object 
 3   transmission  99187 non-null  object 
 4   mileage       99187 non-null  object 
 5   fuelType      99187 non-null  object 
 6   mpg           99187 non-null  float64
 7   engineSize    99187 non-null  float64
dtypes: float64(3), object(5)
memory usage: 6.8+ MB


##Combinando modelos

In [119]:
a = df_car.model.value_counts().reset_index()

In [120]:
a.columns = ['model', 'qtd']

In [121]:
finish = False
limit_s = 3000
limit_i = 2000

group_car = 1
find_start = 0
while a.qtd.iloc[find_start] > limit_i:
  find_start +=1

previous = find_start
actual = find_start + 2

while finish is False:
  if (a.qtd.iloc[previous:actual].sum() < limit_s) & (a.qtd.iloc[previous:actual].sum() >= limit_i):
    modelos = a.model.iloc[previous:actual].values
    
    for modelo in modelos:
      df_car.model.replace(modelo, 'car' + str(group_car), inplace=True)#loc[df_car.model==modelo] = 'car' + str(group_car)
    
    group_car +=1
    previous = actual
    actual += 2
  else:
    if (a.qtd.iloc[previous:a.shape[0]].sum() < limit_s) & (a.qtd.iloc[previous:a.shape[0]].sum() >= limit_i):
      modelos = a.model.iloc[previous:a.shape[0]].values
      for modelo in modelos:
        df_car.model.replace(modelo, 'car' + str(group_car), inplace=True)
      finish=True

    elif a.qtd.iloc[previous:actual].sum() <= limit_i:
      actual +=1
    elif a.qtd.iloc[previous:actual].sum() > limit_s:
      previous +=1
      actual +=1
  
  if actual > a.shape[0]:
    finish=True

##Combinando anos

In [126]:
df_car.year = df_car.year.astype(int)
df_car.year = df_car.year.astype(str)

In [127]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 0 to 110244
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99187 non-null  object 
 1   year          99187 non-null  object 
 2   price         99187 non-null  object 
 3   transmission  99187 non-null  object 
 4   mileage       99187 non-null  object 
 5   fuelType      99187 non-null  object 
 6   mpg           99187 non-null  float64
 7   engineSize    99187 non-null  float64
dtypes: float64(2), object(6)
memory usage: 6.8+ MB


In [130]:
df_car.loc[df_car.year=='2060'].index

Int64Index([110006], dtype='int64')

In [132]:
df_car.drop(index=df_car.loc[df_car.year=='2060'].index, inplace=True)

In [134]:
b = df_car.year.value_counts().reset_index()

In [136]:
b.columns = ['year', 'qtd']

In [142]:
anos = b.year.iloc[4:].values
for ano in anos:
        df_car.year.replace(ano, 'outros', inplace=True)

In [143]:
df_car.year.value_counts()

2019      26771
2017      21733
outros    21168
2016      15850
2018      13664
Name: year, dtype: int64

#testes

In [144]:
df_car.head()

,model,year,price,transmission,mileage,fuelType,mpg,engineSize
0,car1,2017,10550,Manual,25250,Petrol,54.3,1.4
1,car14,2018,8200,Manual,1264,Petrol,67.3,1.0
2,car1,2019,15650,Automatic,6825,Diesel,67.3,2.0
3,car14,outros,14000,Automatic,28431,Diesel,51.4,2.0
4,car9,2019,18350,Manual,10912,Petrol,40.9,1.5
